## Create Multi-Layer NN model using tensorflow

So I can adjust the parameters as I want...

In [27]:
import sys
sys.path.append("./src") # append to system path

from sklearn import cross_validation
from sklearn.metrics import r2_score
from sklearn.metrics import mean_absolute_error
from sklearn.decomposition import PCA
from sklearn.preprocessing import Normalizer
from sklearn.preprocessing import StandardScaler

import numpy as np
import pandas as pd
import tensorflow as tf

import matplotlib.pyplot as plt
from matplotlib import style
from matplotlib.patches import Rectangle
style.use('ggplot')

In [28]:
def load_lcia_data(descs_p, target_p):
    X = pd.read_csv(descs_p,header=0,index_col=None)
    X = X.fillna(0)
    y = pd.read_csv(target_p,header=0,index_col=None)
    return X.values,y.values

def mre(true_y,pred_y):
    ## Note: does not handle mix 1d representation
    #if _is_1d(y_true): 
    #    y_true, y_pred = _check_1d_array(y_true, y_pred)

    return np.mean(np.abs((true_y - pred_y) / true_y)) * 100

### Load Training data
The training data has 156 chemicals now. The rest 10 chemicals are test data
We also split the training and validation data here
We use smaller set (10%) to be the valdiation set (16 chemicals), as the limited size of training chemicals.

In [29]:
descs_p = '../data/descs/train/descs_Mar08_3839_train.csv'
target_p = '../data/target/train/ecosystemquality_train.csv'
X,y = load_lcia_data(descs_p, target_p)

trn_X, val_X, trn_y, val_y = cross_validation.train_test_split(
    X, y, test_size=0.1, random_state=3)

In [30]:
print trn_X.shape, trn_y.shape

(140, 3839) (140, 1)


### Load Testing data

In [31]:
descs_tst = '../data/descs/test/descs_Mar08_3839_test.csv'
target_tst = '../data/target/test/ecosystemquality_test.csv'
tst_X,tst_y = load_lcia_data(descs_tst, target_tst)

### Data Preprocessing
Normalization + PCA or Just Normalization

### Just Scaler

In [32]:
## Standard Scaler
this_scaler = StandardScaler()
trn_X = this_scaler.fit_transform(trn_X)
val_X = this_scaler.transform(val_X)
tst_X = this_scaler.transform(tst_X)

In [33]:
print trn_X.shape
print tst_X.shape
print np.mean(trn_X,0),np.std(trn_X,0)

(140, 3839)
(10, 3839)
[ -1.39123274e-16  -7.31161163e-16   5.43612774e-16 ...,   0.00000000e+00
   2.02219194e-17   8.56457762e-17] [ 1.  1.  1. ...,  0.  1.  1.]


##  PCA

In [34]:
### PCA, don't run them together
# normalize the data first
pca = PCA(n_components = 60)

trn_X = pca.fit_transform(trn_X)
val_X = pca.transform(val_X)
tst_X = pca.transform(tst_X)

In [35]:

print trn_X.shape, tst_X.shape
print(reduce(lambda x,y:x+y,pca.explained_variance_ratio_))

(140, 60) (10, 60)
0.955873062683


In [36]:
plt.scatter(trn_X[:,0],trn_X[:,1])
plt.scatter(tst_X[:,0],tst_X[:,1])
plt.show()

In [37]:
n, bins, patches = plt.hist(trn_y, 50, normed=1, facecolor='green', alpha=0.75)
plt.show()

### Take the Log Scale of Training Target

In [38]:
trn_y = np.log(trn_y)
val_y = np.log(val_y)
tst_y = np.log(tst_y)

n, bins, patches = plt.hist(tst_y,50)
plt.show()

### Build the model

In [63]:
def init_weights(shape):
    weights = tf.random_normal(shape,stddev = 0.1)
    return tf.Variable(weights)

def bias_variable(shape):
    initial = tf.constant(0.1, shape=shape)
    return tf.Variable(initial)

num_descs = trn_X.shape[1]
num_target = trn_y.shape[1]

print num_descs,num_target

60 1


In [64]:
##### 
##Define model structure

X = tf.placeholder(tf.float32,shape=[None,num_descs])
y = tf.placeholder(tf.float32,shape=[None,num_target])

# First layer
w1 = init_weights((num_descs,128)) 
b1 = bias_variable([128])
l1 = tf.add(tf.matmul(X,w1),b1)
l1 = tf.nn.sigmoid(l1)

# Second layer
w2 = init_weights((128,128)) 
b2 = bias_variable([128])
l2 = tf.add(tf.matmul(l1,w2),b2)
l2 = tf.nn.sigmoid(l2)

# # Third layer
# w3 = init_weights((16,16)) 
# b3 = bias_variable([16])
# l3 = tf.add(tf.matmul(l2,w3),b3)
# l3 = tf.nn.sigmoid(l3)

# # Fourth layer
# w4 = init_weights((64,64)) 
# b4 = bias_variable([64])
# l4 = tf.add(tf.matmul(l3,w4),b4)
# l4 = tf.nn.sigmoid(l4)

# # Fifth layer
# w5 = init_weights((64,64)) 
# b5 = bias_variable([64])
# l5 = tf.add(tf.matmul(l4,w5),b5)
# l5 = tf.nn.sigmoid(l5)

# Output layer
w_out = init_weights((128,num_target))
b_out = bias_variable([num_target])
l_out = tf.matmul(l2,w_out) + b_out #no nonlinarity

pred = l_out

In [65]:
#static parameters
BATCH_SIZE = 1
BETA = 0.01 #regularization weights

#Define loss and optimizer 
#Add regularization term
# regularizers = tf.nn.l2_loss(w1) + tf.nn.l2_loss(w2) + tf.nn.l2_loss(w3) + tf.nn.l2_loss(w_out)
regularizers = tf.nn.l2_loss(w1) + tf.nn.l2_loss(w2) + tf.nn.l2_loss(w_out)
cost = tf.reduce_mean(tf.square(pred - y) + BETA*regularizers)

#Gridient Descent Optimizer
optimizer = tf.train.AdagradOptimizer(learning_rate = 0.005).minimize(cost)


# Initializing the variables
init = tf.global_variables_initializer()

## Training

In [66]:
%matplotlib auto
#Start Training
costs=[]

#save the model
saver = tf.train.Saver()


with tf.Session() as sess:
    sess.run(init)
    for epoch in range(200):
        for i in range(0, len(trn_X),BATCH_SIZE):
            _, c = sess.run([optimizer,cost], feed_dict={X:trn_X[i:i+BATCH_SIZE], y:trn_y[i:i+BATCH_SIZE]})
        
        trn_score = r2_score(np.exp(trn_y),np.exp(sess.run(pred, feed_dict={X:trn_X, y:trn_y})))
        val_score = r2_score(np.exp(val_y),np.exp(sess.run(pred, feed_dict={X:val_X, y:val_y})))     
        val_mre = mre(np.exp(val_y),np.exp(sess.run(pred,feed_dict={X:val_X,y:val_y})))
        
        costs.append(val_score)
        if epoch % 50 == 0:
            print("Epoch = %d,Cost = %.2f,Training Accuracy = %.2f, Validation Accuracy = %.2f, Validation MRE =%.2f" % (epoch + 1,c,trn_score,val_score,val_mre))
  
    # final pred on the validation set
    final_pred_val = sess.run(pred,feed_dict={X:val_X})
    # prediction on the testing set
    final_pred_test = sess.run(pred,feed_dict={X:tst_X})
    
    # convert it back 
    
    for (y,y_hat) in zip(np.exp(tst_y),np.exp(final_pred_test)):
        print y,y_hat
    
    
    plt.plot(costs)
    plt.show()
    
    save_path = saver.save(sess, "../nets/ecosystemquality/ecosystemquality_Apr4.ckpt")
    print("Model saved in file: %s" % save_path)

Using matplotlib backend: TkAgg
Epoch = 1,Cost = 1.51,Training Accuracy = 0.02, Validation Accuracy = -0.26, Validation MRE =129.32
Epoch = 51,Cost = 0.92,Training Accuracy = 0.72, Validation Accuracy = 0.30, Validation MRE =48.94
Epoch = 101,Cost = 0.73,Training Accuracy = 0.82, Validation Accuracy = 0.39, Validation MRE =50.08
Epoch = 151,Cost = 0.65,Training Accuracy = 0.87, Validation Accuracy = 0.41, Validation MRE =50.53
[ 0.00012772] [ 0.00017884]
[ 0.00012991] [  3.64186562e-05]
[ 0.00011437] [ 0.00010507]
[ 0.00040355] [ 0.00016049]
[ 0.00036248] [ 0.00025309]
[  6.37000000e-05] [  3.55146440e-05]
[  8.74000000e-05] [ 0.00055158]
[  1.17000000e-05] [  2.98560317e-05]
[  9.31000000e-05] [  6.64469771e-05]
[  2.41000000e-05] [  2.84197358e-05]
Model saved in file: ../nets/ecosystemquality/ecosystemquality_Apr4.ckpt


In [67]:
# Convert the value back, remember only run this once 
tst_y = np.exp(tst_y) 
final_pred_test = np.exp(final_pred_test) 

In [68]:
MRE_this = mre(tst_y, final_pred_test)
R2_this = r2_score(tst_y, final_pred_test)
print R2_this
MRE_label = 'MRE: ' + str(round(MRE_this,2))

fig = plt.figure()
ax = fig.add_subplot(111)
est = plt.plot(tst_y, final_pred_test,'o', label='estimated values')

max_val = max(max(tst_y),max(final_pred_test))
plt.ylim([0,max_val])
plt.xlim([0,max_val])

thisLine = plt.plot(np.append(0,max_val), np.append(0,max_val), label='perfect prediction line')

plt.plot([],[],linewidth=0, label=MRE_label)
plt.legend(loc='upper left')
plt.show()

-0.867792444697
